In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from textblob import TextBlob

#Data preprocessing to removes @usernames,urls,symbols and makes all text lowercase
def preprocess_text(text):
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', text)
    text = re.sub('@[^\s]+','', text)
    text = text.lower().replace("ё", "е")
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+',' ', text)
    text = re.sub(' +',' ', text)
    return text.strip()

In [2]:
#load data from csv/excel/json
data=pd.read_excel('tweets.xlsx') 
#data.head()

In [3]:
text = [preprocess_text(t) for t in data.iloc[:]['full_text']]

#storing preprocessed texts in a separate column 
data["processed_text"]= text

#example
print(data.iloc[0]['full_text'])
print(data.iloc[0]["processed_text"])

@ClareTyne @mesainy @HKrassenstein @realDonaldTrump @NYCMayor @JoeBiden Yep.  Torturing that guy because the Dems donâ€™t care.
yep torturing that guy because the dems don t care


In [4]:
#data.head(2)

In [5]:
#Tokenizing 
tokens = [nltk.word_tokenize(t) for t in data.iloc[:]['processed_text']]
#print(tokens)

In [6]:
#Textblob

#calling the Textblob function for each text
blobs = [TextBlob(t) for t in data.iloc[:]["processed_text"]]

selected = data[["id_str","full_text","processed_text"]]
df=selected.copy()

#getting the polarity and subjectivity of the blob
polarity=[]
for i in range(data.shape[0]):
    polarity.append(blobs[i].sentiment)
    
df["Polarity/subjectivity"]= polarity

sentiment= []
for j in range(data.shape[0]):
    if polarity[j][0]>0:
        sentiment.append("Positive")
    elif polarity[j][0]<0:
        sentiment.append("Negative")
    else:
        sentiment.append("Neutral")
        
df["Sentiment"]= sentiment
pd.options.display.max_colwidth = 150
df[["full_text","Polarity/subjectivity","Sentiment"]]

,full_text,Polarity/subjectivity,Sentiment
0,@ClareTyne @mesainy @HKrassenstein @realDonaldTrump @NYCMayor @JoeBiden Yep. Torturing that guy because the Dems donâ€™t care.,"(0.0, 0.0)",Neutral
1,"""Arizona reports record spike in new coronavirus cases &amp; deaths ahead of Penceâ€™s visit"" His megachurch stunt took place 5 days ago in Phoeni...","(0.13636363636363635, 0.45454545454545453)",Positive
2,@JoeBiden Debate President Trump. PROVE you donâ€™t have dementia. #DementiaJoeCantDebate #JoeBidenScaredToDebate,"(0.0, 0.0)",Neutral
3,@realDonaldTrump You disrespected our soldiers in Afganistan so your not and better.,"(0.5, 0.5)",Positive
4,@PamelaStovall6 @ChuckGrassley @realDonaldTrump Democrats are always saving @GOP dinosaurs like grassley after they fvck up,"(0.0, 0.0)",Neutral
5,@Jorgensen4POTUS @RealSpikeCohen Just found out about you and so far I love your policies and what you have to say. Ofc I still need to read more ...,"(0.35555555555555557, 0.6944444444444443)",Positive
6,"@95YSIUXcGfGGXnj @VijayNeupane1 @AngelaBelcamino @realDonaldTrump Dude, get a grip","(0.0, 0.0)",Neutral
7,@jonathanvswan This is like the 4th or 5th time this story has come out since Trump took office.,"(0.0, 0.0)",Neutral
8,@ExcaliberXX @HormanTodd @realDonaldTrump After murders of young people?,"(0.1, 0.4)",Positive
9,@jonathanvswan @jaketapper The Impeached @realDonaldTrump will always blame others.,"(0.0, 0.0)",Neutral
